In [ ]:
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')

def folder(f: str):
  fp = "/content/drive/MyDrive/optimization-techniques/" + f
  Path(fp).mkdir(parents = True, exist_ok = True)
  return fp

# Benchmark functions
The complete list of functions available can be found at: https://gitlab.com/luca.baronti/python_benchmark_functions




In [ ]:
!pip3 install benchmark_functions
import benchmark_functions as bf
import numpy as np
from scipy.optimize import minimize, rosen
from matplotlib import pyplot as plt

class OptFun():
    def __init__(self, wf):
        self.f = wf
        self.history = []
        
    def __call__(self, x0):
        self.history.append(x0.copy())
        return self.f(x0)

    def minima(self):
        return self.f.minimum()
        
    def bounds(self):
        return self._convert_bounds(self.f.suggested_bounds())

    def in_bounds(self, p):
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        for i in range(len(p)):
            if p[i]<bounds_lower[i] or p[i]>bounds_upper[i]:
                return False
        return True

    def heatmap(self, fn = None):
        plt.clf()
        resolution = 50
        fig = plt.figure()
        fig.canvas.set_window_title('Benchmark Function: '+self.f._name)
        fig.suptitle(self.f._name)
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
        if self.f._n_dimensions>1:
            y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
            X, Y = np.meshgrid(x, y)
            Z = np.asarray([[self.f((X[i][j],Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

        plt.contour(x,y,Z,15,linewidths=0.5,colors='k') # hight lines
        plt.contourf(x,y,Z,15,cmap='viridis', vmin=Z.min(), vmax=Z.max()) # heat map
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('z')
        if len(self.history)>0:	# plot points
            xdata = [x[0] for x in self.history]
            ydata = [x[1] for x in self.history]
            plt.plot(xdata, ydata, "or-", markersize=2, linewidth=2)
        if fn is None:
            plt.show()
        else:
            plt.savefig(fn, dpi=400)

    def plot(self, fn =None):
        plt.clf()
        showPoints = [x for x in self.history if self.in_bounds(x)]
        xp = [h for h in showPoints]
        values = [self.f(v) for v in xp]
        min = self.f.minimum().score
        bounds = self.bounds()
        Dtmp = [np.linspace(bounds[i][0], bounds[i][1], num=1000) for i in range(len(bounds))]
        D = []
        for i in range(1000):
            D.append([Dtmp[j][i] for j in range(len(bounds))])
        C = []
        for i in range(1000):
            C.append(self.f([Dtmp[j][i] for j in range(len(bounds))]))
        plt.plot(D, C,  label = "function")
        plt.plot(xp, values,  '.', color="r", label="points")
        
        plt.legend()
        if fn is None:
            plt.show()
        else:
            plt.savefig(fn, dpi=400)
            
    def trend(self, fn = None):
        plt.clf()
        showPoints = [x for x in self.history if self.in_bounds(x)]
        values = [self.f(list(v)) for v in showPoints]
        min = self.f.minimum().score
        plt.plot(values)
        plt.axhline(min, color="r", label="optimum")
        plt.legend()
        if fn is None:
            plt.show()
        else:
            plt.savefig(fn, dpi=400)

    def _convert_bounds(self, bounds):
        new_bounds= []
        for i in range(len(bounds[0])):
            new_bounds.append((bounds[0][i], bounds[1][i]))
        return new_bounds

    def magic(self, resolution=50, file = None):
      bounds_lower, bounds_upper = self.f.suggested_bounds()

      x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
      y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
      X, Y = np.meshgrid(x, y)
      Z = np.asarray([[self.f((X[i][j],Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

      fig = plt.figure()
      fig.canvas.set_window_title('Benchmark Function: '+self.f._name)
      fig.suptitle(self.f._name)

      ax = plt.axes(projection='3d')

      ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')

      ax.view_init(60, 35)
      ax.set_zlabel('z')
      ax.set_xlabel('x')
      ax.set_ylabel('y')

      if file is None:
        plt.show()
      else:
        plt.savefig(file, dpi=400)


def powell(f: OptFun, x0, maxiter: int):
    """
    Optimizes a function by using the Powell algorithm.

    - f: function to optimize, an instance of OptFun
    - x0: starting point for the search process
    - maxiter: maximum number of iterations
    """
    bounds = f.bounds()
    results = minimize(fun=f, x0=list(x0), method="powell", bounds=bounds, 
                       options={"ftol":1e-4,
                                "maxfev": None,
                                "maxiter": maxiter,
                                "return_all":True})
    return results

def nelder_mead(f: OptFun, x0, maxiter: int):
    """
    Optimizes a function by using the Nelder-Mead algorithm.

    - f: function to optimize, an instance of OptFun
    - x0: starting point for the search process
    - maxiter: maximum number of iterations
    """
    bounds = f.bounds()
    return minimize(
        f,
        x0,
        method='Nelder-Mead',
        tol=None,
        bounds=bounds,
        options={
            "maxfev": None,
            "maxiter": maxiter,
            'disp': False,
            'return_all': True,
            'initial_simplex': None,
            'xatol': 0.000,
            'fatol': 0.000,
            'adaptive': False
        }
    )

In [ ]:
drive = folder("fun/")

for (name, fn) in [
    ("Hypershpere", bf.Hypersphere()), 
    ("DeJong3", bf.DeJong3()), 
    ("PichenyGoldsteinAndPrice", bf.PichenyGoldsteinAndPrice())
    ]:
  OptFun(fn).magic(file = drive + name + ".svg")